In [14]:
import time
import pymysql
import datetime

# 打开数据库连接
db = pymysql.connect("192.168.1.94", "dataUser94",
                     "94dataUser@2020", "saicnqms")

# 使用cursor()方法获取操作游标
cursor = db.cursor()

the_day_before_yesterday = time.strftime(
    "%Y-%m", time.localtime()) + '-' + str(int(time.strftime("%d", time.localtime())) - 3)
last_month = datetime.datetime.now().month - 1
last_month = f"0{str(last_month)}" if last_month < 10 else last_month
last_month = str(datetime.datetime.now().year) + '-' + str(last_month) + '-01'
last_month = '2020-10-01'
print(the_day_before_yesterday, last_month)


2020-10-24 2020-10-01


In [15]:
# SQL 查询语句
# 总url
try:
    #     获取表头
    cursor.execute(
        '''select *  from autohome_luntan_allurl;''')
    cols1 = [i[0] for i in cursor.description]
    print(cols1)
    # 执行SQL语句
    sql1 = "SELECT posted_time,user_car,brand,factory,tiezi_url,isvideo FROM autohome_luntan_allurl WHERE posted_time <= '{}' AND  posted_time >= '{}'".format(
        the_day_before_yesterday, last_month)
    cursor.execute(sql1)
    # 获取所有记录列表
    results1 = cursor.fetchall()
    print(len(results1))
#     print(results1)
except:
    print("总URL表查询失败")

['_id', 'grabtime', 'posted_time', 'isvideo', 'car_id', 'user_car', 'page', 'brand', 'factory', 'tiezi_url', 'status']
88811


In [5]:
# 已经抓取的url
try:
    #     获取表头
    cursor.execute('''select *  from luntan_all_copy;''')
    cols2 = [i[0] for i in cursor.description]
    print(cols2)
    # 执行SQL语句
    sql2 = "SELECT posted_time,user_car,brand,factory,url FROM luntan_all_copy WHERE posted_time <= '{}' AND posted_time >= '{}' AND information_source = 'autohome'".format(
        the_day_before_yesterday, last_month)
    cursor.execute(sql2)
    results2 = cursor.fetchall()
    print(len(results2))
except:
    print("luntan_all_copy表查询失败")

# 关闭数据库连接

db.close()

['id', 'brand', 'click_num', 'content', 'content_num', 'factory', 'grabtime', 'information_source', 'parsetime', 'posted_time', 'province', 'region', 'reply_num', 'statusplus', 'title', 'url', 'user_car', 'user_name', 'create_time']
78683


In [16]:
import pandas as pd
import numpy as np

df1 = pd.DataFrame(np.array(results1), columns=['posted_time','user_car','brand','factory','url','isvideo'])
print('总url表去重前：', len(df1))
df1 = df1.drop_duplicates('url')
print('总url表去重后：', len(df1))
df1['url'] = df1['url'].map(lambda x: x.replace('https', 'http'))

print(df1.head())

总url表去重前： 88811
总url表去重后： 47895
           posted_time   user_car brand factory  \
0  2020-10-23 17:33:50       创界论坛   雪佛兰    上汽通用   
1  2020-10-23 13:49:33       创界论坛   雪佛兰    上汽通用   
2  2020-10-23 11:49:05       创界论坛   雪佛兰    上汽通用   
3  2020-10-22 20:23:38       创界论坛   雪佛兰    上汽通用   
4  2020-10-20 18:55:28  Orlando论坛   雪佛兰    上汽通用   

                                                 url isvideo  
0  http://club.autohome.com.cn/bbs/thread/4f8b719...       0  
1  http://club.autohome.com.cn/bbs/thread/85d2954...       0  
2  http://club.autohome.com.cn/bbs/thread/d12654b...       0  
3  http://club.autohome.com.cn/bbs/thread/13a9ab9...       1  
4  http://club.autohome.com.cn/bbs/thread/6528409...       0  


In [7]:
df2 = pd.DataFrame(np.array(results2), columns=['posted_time','user_car','brand','factory','url'])
print('已抓取表去重前：', len(df2))
df2 = df2.drop_duplicates('url')
print('已抓取表去重后：', len(df2))
df2['url'] = df2['url'].map(lambda x: x.replace('https', 'http'))

print(df2.head())

已抓取表去重前： 78683
已抓取表去重后： 38300
          posted_time   user_car brand factory  \
0 2020-10-12 06:55:40  荣威RX5 MAX    荣威   上汽乘用车   
1 2020-10-12 06:56:27  荣威RX5 MAX    荣威   上汽乘用车   
2 2020-10-12 06:59:41  荣威RX5 MAX    荣威   上汽乘用车   
3 2020-10-09 22:13:11      迈锐宝XL   雪佛兰    上汽通用   
4 2020-10-10 13:03:49  荣威RX5 MAX    荣威   上汽乘用车   

                                                 url  
0  http://club.autohome.com.cn/bbs/thread/c2480a0...  
1  http://club.autohome.com.cn/bbs/thread/9778b8b...  
2  http://club.autohome.com.cn/bbs/thread/ce6d9bf...  
3  http://club.autohome.com.cn/bbs/thread/7442983...  
4  http://club.autohome.com.cn/bbs/thread/dc20e22...  


In [17]:
df1 = df1.append(df2)
df1 = df1.append(df2)
df1.head()

,posted_time,user_car,brand,factory,url,isvideo
0,2020-10-23 17:33:50,创界论坛,雪佛兰,上汽通用,http://club.autohome.com.cn/bbs/thread/4f8b719...,0
1,2020-10-23 13:49:33,创界论坛,雪佛兰,上汽通用,http://club.autohome.com.cn/bbs/thread/85d2954...,0
2,2020-10-23 11:49:05,创界论坛,雪佛兰,上汽通用,http://club.autohome.com.cn/bbs/thread/d12654b...,0
3,2020-10-22 20:23:38,创界论坛,雪佛兰,上汽通用,http://club.autohome.com.cn/bbs/thread/13a9ab9...,1
4,2020-10-20 18:55:28,Orlando论坛,雪佛兰,上汽通用,http://club.autohome.com.cn/bbs/thread/6528409...,0


In [18]:
df0 = df1.drop_duplicates(subset=['url'], keep=False)
df0.shape

(10698, 6)

In [19]:
df0.head()

,posted_time,user_car,brand,factory,url,isvideo
0,2020-10-23 17:33:50,创界论坛,雪佛兰,上汽通用,http://club.autohome.com.cn/bbs/thread/4f8b719...,0
1,2020-10-23 13:49:33,创界论坛,雪佛兰,上汽通用,http://club.autohome.com.cn/bbs/thread/85d2954...,0
2,2020-10-23 11:49:05,创界论坛,雪佛兰,上汽通用,http://club.autohome.com.cn/bbs/thread/d12654b...,0
3,2020-10-22 20:23:38,创界论坛,雪佛兰,上汽通用,http://club.autohome.com.cn/bbs/thread/13a9ab9...,1
4,2020-10-20 18:55:28,Orlando论坛,雪佛兰,上汽通用,http://club.autohome.com.cn/bbs/thread/6528409...,0


In [23]:
df0 = df0.drop_duplicates('url')
df0.shape

(10698, 6)

In [24]:
#漏抓url写入mongo
# from sqlalchemy import create_engine

# connect = create_engine('mysql+pymysql://94dataUser@2020:dataUser94@192.168.1.94:3306/saicnqms?charset=utf-8')
# pd.io.sql.to_sql()

In [21]:
#漏抓url写入mongo
import pymongo
mongo_client = pymongo.MongoClient('192.168.2.149', 27017)
db = mongo_client['luntan']
mycol = db['autohome_luntan_lost']
data = df0.to_dict(orient='record')
mycol.insert_many(data)

In [5]:
import requests
import json
from lxml import etree
headers = {
    'Referer': 'https://club.autohome.com.cn/',
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36"
}
response = requests.get(url = 'https://club.autohome.com.cn/frontapi/topics/getByBbsId?pageindex=1&pagesize=100&bbs=c&bbsid=533&fields=topicid%2Ctitle%2Cpost_memberid%2Cpost_membername%2Cpostdate%2Cispoll%2Cispic%2Cisrefine%2Creplycount%2Cviewcount%2Cvideoid%2Cisvideo%2Cvideoinfo%2Cqainfo%2Ctags%2Ctopictype%2Cimgs%2Cjximgs%2Curl%2Cpiccount%2Cisjingxuan%2Cissolve%2Cliveid%2Clivecover%2Ctopicimgs&orderby=topicid-',headers=headers)
response.encoding = response.apparent_encoding
html = etree.HTML(response.text)
json_data = json.loads(response.text)
print(json_data)

{'returncode': 0, 'message': '', 'result': {'pageindex': 1, 'pagecount': 15, 'rowcount': 1478, 'currpagerowcount': 0, 'list': [{'topicid': 91028654, 'title': '车门有一小块凹陷,没伤车漆,怎么修复?', 'post_memberid': 199375672, 'post_membername': 'PP小朋友', 'postdate': '2020-10-25 23:13:57', 'ispic': 0, 'ispoll': 18, 'isrefine': 0, 'replycount': 4, 'viewcount': 0, 'videoid': '', 'isvideo': 0, 'videoinfo': None, 'qainfo': {'topicid': 91028654, 'issolve': False, 'ishigh': False, 'confirmreplyid': 0, 'summary': ''}, 'tags': [], 'topictype': '问', 'imgs': '', 'jximgs': '', 'url': 'http://club.autohome.com.cn/bbs/thread/ac29bb177ee5eb48/91028654-1.html', 'piccount': 0, 'isjingxuan': 0, 'issolve': 0, 'liveid': None, 'livecover': None}, {'topicid': 90921495, 'title': '均速行驶能省油吗？', 'post_memberid': 10686541, 'post_membername': '芝罘人', 'postdate': '2020-10-21 20:31:47', 'ispic': 0, 'ispoll': 18, 'isrefine': 0, 'replycount': 1, 'viewcount': 0, 'videoid': '', 'isvideo': 0, 'videoinfo': None, 'qainfo': {'topicid': 909214